# Create and Use Asset
This notebook leverages the Catalog API to create and use an asset.

Before creating an asset, a schema must first be defined. Afterwards, assets can be created according to this defined schema.

In the Catalog API, a schema for an asset is defined using a type definition. For this use case, we create a type definition for a new asset type named *publicDataSet*.

With the schema defined, assets and relationships can be created using instances endpoints. In the Catalog AP, an instance of an asset type is known as an entity. The scenario shows both how to create a single asset and how to use an archive to create or update assets and relationships. The relationship in this scenario is adding a contact to an asset.

The assets are queried using both a view and a filter on the instances endpoint.

NOTE: Please make sure you run [Imports and Global Variables](#imports) before executing anything else in this notebook. Also ensure you are authenticated by running [Get Access Token](#authentication).


## Imports and Global Variables <a id='imports'></a>
Run this cell before any of the others as it imports packages and sets variables that will be used throughout the notebook.

In [ ]:
import requests
import json

sasserver = "https://your_server"
username = "your_username"

print (sasserver)
print (username)

# Get Access token <a id='authentication'></a>

In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload={
    'grant_type': 'password',
    'username': username,
    'password': "your_password"
}
files=[

]
headers = {
  'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)

response = response.json()
access_token = response["access_token"]

# Create type definition


In [ ]:
url = sasserver + "/catalog/definitions"

payload = json.dumps({"definitionType": "entity",
                      "description": "Public Data Set",
                      "id": "3eb3719a-2c62-4889-b66e-abe6cb24c7a8",
                      "name": "publicDataSet",
                      "platformTypeName": "publicDataSet",
                      "label": "Public Data Set",
                      "version": 2,
                      "baseType": "asset",
                      "metaCategory": "PRIMARY",
                      "attributeDefinitions": {
                          "externalLink": {
                              "description": "External URL of the resource",
                              "name": "externalLink",
                              "label": "Resource Link",
                              "type": "string"
                          },
                          "usabilityRating": {
                              "name": "usabilityRating",
                              "label": "Usability Rating",
                              "type": "number"
                          },
                          "keywords": {
                              "name": "keywords",
                              "label": "Keywords",
                              "description": "Terms used to search and describe the data set.",
                              "type": "string"
                          },
                          "license": {
                              "name": "license",
                              "label": "license",
                              "type": "string"
                          }
                      }
                     })
headers = {
  'Content-Type': 'application/vnd.sas.metadata.definition.entity+json;version=2',
  'Accept': 'application/vnd.sas.metadata.definition.entity+json;version=2',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))
type_definition_id = response["id"]
type_definition_name = response["name"]


# Create entity instance

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "version": 1,
  "id": "97c78bcd-cd40-4f22-a1c6-dc3e49c5244a",
  "instanceType": "entity",
  "definition": "publicDataSet",
  "name": "countries-of-the-world-2023",
  "description": "A Comprehensive Dataset Empowering In-Depth Analysis and Cross-Country Insights",
  "label": "Global Country Information Dataset 2023",
  "attributes": {
    "externalLink": "https://www.kaggle.com/datasets/nelgiriyewithana/countries-of-the-world-2023",
    "usabilityRating": 1,
    "keywords": "education,social science,computer science,data visualization,data analytics,classification",
    "license": "Attribution 4.0 International (CC BY 4.0)"
  },
  "resourceId": "/catalog/instances/97c78bcd-cd40-4f22-a1c6-dc3e49c5244a"
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.entity+json',
  'Accept': 'application/vnd.sas.metadata.instance.entity+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))
entity_id = response["id"]


# Create entity for principal to use for contact

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "version": 1,
  "instanceType": "entity",
  "definition": "principal",
  "definitionId": "9438c63d-ac1d-4c96-a6f4-087463adef41",
  "name": username,
  "type": "principal",
  "attributes": {
    "principalType": "user"
  },
  "resourceId": "/identities/users/etladm"
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.entity+json',
  'Accept': 'application/vnd.sas.metadata.instance.entity+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
response = response.json()
print(json.dumps(response, indent=2))

# Get principal for contact

In [ ]:
url = sasserver + "/catalog/instances?filter=and(eq(type,'principal'),eq(name,'" + username + "'))"

payload = ""
headers = {
  'Accept-Item': 'application/vnd.sas.metadata.instance.entity+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))
principal_id = response['items'][0]['id']

#  Create/Update entity and relationship with instance archive

In [ ]:
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime

now = datetime.now()
stamp = mktime(now.timetuple())
modified_time = format_date_time(stamp)

url = sasserver + "/catalog/instances?onConflict=update"

payload = json.dumps({
  "entities": [
    {
      "version": 1,
      "id": "749c06e2-3074-4a6b-a2de-7a42876f4db0",
      "instanceType": "entity",
      "definition": "publicDataSet",
      "name": "global-data-on-sustainable-energy",
      "description": "If you found this data helpful, a vote is appreciated u2764ufe0f.  Do share your feedback in the discussion section.nn# Key Features:n&gt;- **Entity**: The name of the country or region for which the data is reported.n- **Year**: The year for which the data is reported, ranging from 2000 to 2020.n- **Access to electricity (% of population)**: The percentage of population with access to electricity.n- **Access to clean fuels for cooking (% of population)**: The percentage of the population with primary reliance on clean fuels.n- **Renewable-electricity-generating-capacity-per-capita**: Installed Renewable energy capacity per personn- **Financial flows to developing countries (US $)**: Aid and assistance from developed countries for clean energy projects.n- **Renewable energy share in total final energy consumption (%)**: Percentage of renewable energy in final energy consumption.n- **Electricity from fossil fuels (TWh)**: Electricity generated from fossil fuels  ..(truncated)",
      "attributes": {
        "externalLink": "https://www.kaggle.com/datasets/anshtanwar/global-data-on-sustainable-energy",
        "usabilityRating": 1,
        "keywords": "education,energy,computer science,data visualization,classification",
        "license": "Attribution 4.0 International (CC BY 4.0)"
      },
      "resourceId": "/catalog/instances/749c06e2-3074-4a6b-a2de-7a42876f4db0"
    }
  ],
  "relationships": [
    {
      "id": "f09b2631-665f-4f16-905d-a172b7b45085",
      "creationTimeStamp": "0001-01-01T00:00:00Z",
      "modifiedTimeStamp": "0001-01-01T00:00:00Z",
      "instanceType": "relationship",
      "definition": "assetContacts",
      "endpoint1Id": "749c06e2-3074-4a6b-a2de-7a42876f4db0",
      "endpoint2Id": principal_id
    }
  ]
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.archive+json',
  'Accept': 'application/vnd.sas.metadata.instance.archive+json',
  'If-Unmodified-Since': modified_time,
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))

entity2_id = response['entities'][0]['id']

# Get an archive of entities and relationship using a view

In [ ]:
url = sasserver + "/catalog/instances"
payload = json.dumps({
  "query": "match (t)-[r]-(c) return t, r, c",
  "parameters": {
    "t": {
      "id": entity2_id
    }
  }
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.query+json',
  'Accept': 'application/vnd.sas.metadata.instance.archive+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))


#  Get an archive of entities related to principal contact

In [ ]:
url = sasserver + "/catalog/instances"
payload = json.dumps({
  "query": "match (t)-[r]-(c) return c",
  "parameters": {
    "t": {
      "id": principal_id
    }
  }
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.query+json',
  'Accept': 'application/vnd.sas.metadata.instance.archive+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))

# Find entity with filter using type and name

In [ ]:
url = sasserver + "/catalog/instances?filter=and(eq(type,'publicDataSet'),contains(name, 'global'))"

payload = ""
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))

#  Delete entities using archive

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "deletions": [
    entity_id,
    entity2_id
  ]
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.archive+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

response = response.json()
print(json.dumps(response, indent=2))

# Delete a type definition

In [ ]:
url = sasserver + "/catalog/definitions/" + type_definition_id

payload = {}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Type definition deleted successfully")
else:
    print(f"Error. Here is the response:\n{response.json()}")
